The dataset contains six features (first six cloumns) and a target variable (the 7th column) 

First step is to load the dataset to RDD.

In [2]:
from pyspark.sql.types import *

# load the text file to rdd
# The file is located at 'https://www.dropbox.com/s/ynz19sxmjny7fdi/solved_heuristic_1.txt?dl=0'
rdd = sc.textFile('/FileStore/tables/bbtax3gn1457669500982/solved_heuristic_2.txt')
print rdd.count()
print rdd.first()

#  separate the label from the features
rdd1 = rdd.map(lambda l: l.split(","))

In [3]:
# convert the RDD to data frame
schema = StructType([
    StructField('f1', StringType(), True),
    StructField('f2', StringType(), True),
    StructField('f3', StringType(), True),
    StructField('f4', StringType(), True),
    StructField('f5', StringType(), True),
    StructField('f6', StringType(), True),
    StructField('label', StringType(), True)
  ])
df = sqlContext.createDataFrame(rdd1, schema)

In [4]:
#convert the character columns to numeric
df = df.withColumn("f1", df["f1"].cast(IntegerType()))
df = df.withColumn("f2", df["f2"].cast(IntegerType()))
df = df.withColumn("f3", df["f3"].cast(IntegerType()))
df = df.withColumn("f4", df["f4"].cast(IntegerType()))
df = df.withColumn("f5", df["f5"].cast(IntegerType()))
df = df.withColumn("f6", df["f6"].cast(IntegerType()))
df = df.withColumn("label", df["label"].cast(DoubleType()))

In [5]:
df
print df.dtypes
df.describe().show()
df.first()

In [6]:
#use a vector assembler to create features
from pyspark.ml.feature import VectorAssembler

ignore = ['label']
VA = VectorAssembler(
    inputCols=[x for x in df.columns if x not in ignore],
    outputCol='features')

VA1 = VA.transform(df)
#print(VA1.select("features", "target").first())

df1 = (VA1.select("features", "label"))
df1.first()


In [7]:
# divide the data to training and testing
splits = df1.randomSplit([0.8, 0.2], 2569)
train = splits[0].take(5)
test = splits[1].take(5)

print train.pop(1)

In [8]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
# create the layers
# 6 input, 3 intermediate, and 1 output layers
layers = [6, 4, 3, 100]
# create the trainer and set its parameters
#trainer = MultilayerPerceptronClassifier(maxIter=10, layers=layers, blockSize=128, seed=1234)
trainer = MultilayerPerceptronClassifier(maxIter=1)
# train the model
model = trainer.fit(train)
# compute precision on the test set
result = model.transform(test)